# Pose3D
Pose3Base is a class inherited from the ndarray of rthe *numpy* to exted it with the direct compounding ($\oplus$) and the inverse compounding ($\ominus$) operators.

## Direct Compounding ##
Let us consider 2 transformations from A-Frame to B-Frame and from B-Frame to C-Frame respectively:
$$
\begin{split}
\mathbf{{^A}x_B}&=[^Ax_B~~^Ay_B~~^Az_B~~{^A{\psi_B}}]^T\\
\mathbf{{^B}x_C}&=[^Bx_C~~^By_C~~^Bz_C~~{^B\psi_C}]^T
\end{split}
$$

which can be graphycally represented as follows:

<center>
<img src="./img/Compounding.png"
     width="500" />

    Compounding Operation
</center>

the compounding operation is defined as follows:

$$
   \mathbf{{^A}x_{C}}=\mathbf{{^A}x_B} \oplus \mathbf{{^B}x_C}=
        \begin{bmatrix} ^Ax_C~^Ay_C~^Az_C~^A\psi_C\end{bmatrix}^T
$$


In [80]:
import numpy as np
from math import atan2, cos, sin

class Pose3D(np.ndarray):
    """
    Definition of a robot pose in 3 DOF (x, y, yaw). The class inherits from a ndarray.
    This class extends the ndarray with the $oplus$ and $ominus$ operators and the corresponding Jacobians.
    """
    def __new__(cls, input_array):
        """
        Constructor of the class. It is called when the class is instantiated. It is required to extend the ndarry numpy class.

        :param input_array: array used to initialize the class
        :returns: the instance of a Pose3D class object
        """
        assert input_array.shape == (3, 1), "mean must be a 3x1 vector"

        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        # Finally, we must return the newly created object:
        return obj

    def oplus(AxB, BxC):
        """
        Given a Pose3D object *AxB* (the self object) and a Pose3D object *BxC*, it returns the Pose3D object *AxC*.

        .. math::
            \\mathbf{{^A}x_B} &= \\begin{bmatrix} ^Ax_B & ^Ay_B & ^A\\psi_B \\end{bmatrix}^T \\\\
            \\mathbf{{^B}x_C} &= \\begin{bmatrix} ^Bx_C & ^By_C & & ^B\\psi_C \\end{bmatrix}^T \\\\

        The operation is defined as:

        .. math::
            \\mathbf{{^A}x_C} &= \\mathbf{{^A}x_B} \\oplus \\mathbf{{^B}x_C} =
            \\begin{bmatrix}
                ^Ax_B + ^Bx_C  \\cos(^A\\psi_B) - ^By_C  \\sin(^A\\psi_B) \\\\
                ^Ay_B + ^Bx_C  \\sin(^A\\psi_B) + ^By_C  \\cos(^A\\psi_B) \\\\
                ^A\\psi_B + ^B\\psi_C
            \\end{bmatrix}
            :label: eq-oplus3dof

        :param BxC: C-Frame pose expressed in B-Frame coordinates
        :returns: C-Frame pose expressed in A-Frame coordinates
        """

        # TODO: to be completed by the student
        xb_c = BxC[0]
        yb_c = BxC[1]
        wb_c = BxC[2]
        xa_b = AxB[0]
        ya_b = AxB[1]
        wa_b = AxB[2]
        output = Pose3D(np.array([xa_b + xb_c*cos(wa_b)-yb_c*sin(wa_b),
                           ya_b + xb_c*sin(wa_b)+yb_c*cos(wa_b),
                           wa_b + wb_c]
                          ) )
    
        return output

    def ominus(AxB):
        """
        Inverse pose compounding of the *AxB* pose (the self objetc):

        .. math::
            ^Bx_A = \\ominus ^Ax_B =
            \\begin{bmatrix}
                -^Ax_B \\cos(^A\\psi_B) - ^Ay_B \\sin(^A\\psi_B) \\\\
                ^Ax_B \\sin(^A\\psi_B) - ^Ay_B \\cos(^A\\psi_B) \\\\
                -^A\\psi_B
            \\end{bmatrix}
            :label: eq-ominus3dof

        :returns: A-Frame pose expressed in B-Frame coordinates (eq. :eq:`eq-ominus3dof`)
        """

        # TODO: to be completed by the student
        xa_b = AxB[0]
        ya_b = AxB[1]
        wa_b = AxB[2]
        output = Pose3D(np.array([-xa_b*cos(wa_b)- ya_b*sin(wa_b),
                           xa_b*sin(wa_b) - ya_b*cos(wa_b),
                           -wa_b ]
                          ) )

        return output

        pass


as well as the inverse compounding ($\ominus$):

<center>
<img src="./img/InverseCompounding.png"
     width="400" />

    Inverse Compounding Operation
</center>

### Test
Now, let us test the compounding operation. To do it, define the following 2 Poses:
$$\mathbf{^A{x_B}}=\left[1~2~\frac{\pi}{2}\right]^T$$
$$\mathbf{^B{x_C}}=\left[3~4~\pi\right]^T$$
if we compound the 2 poses we get:
$$\mathbf{^A{x_C}}= \mathbf{^A{x_B}} \oplus \mathbf{^B{x_C}}=\left[-3~5~\frac{3\pi}{2}\right]^T$$
Let us program a code to reproduce this behaviour

In [81]:
AxB = Pose3D(np.array([[1],[2] , [np.pi/2]])) # TODO: complete this sentence to define the Pose Transformation as defined above
BxC = Pose3D(np.array([[3], [4], [np.pi]]))# TODO: complete this sentence to  define the Pose Transformation as defined above
AxC = AxB.oplus(BxC)# TODO: complete this sentence to compute the pose transformation from A-Frame to C-Frame
print(BxC[2])
print("AxC=",AxC.T)
print(AxC.shape)

[3.14159265]
AxC= [[-3.          5.          4.71238898]]
(3, 1)


Now let us try the ominus operation:
$$\ominus\mathbf{^A{x_C}}= \left[5~3~\frac{-3\pi}{2}\right]^T$$

In [82]:
CxA= Pose3D(AxC).ominus()# TODO: complete this sentence to compute the pose transformation from C-Frame to A-Frame

print("CxA=",CxA.T)

CxA= [[ 5.          3.         -4.71238898]]
